In [24]:
from sqlalchemy import (
    create_engine, 
    MetaData,
    Table,
    Column,
    Integer,
    String,
    Numeric,
    Boolean,
    DateTime,
    ForeignKey,
    CheckConstraint,
    insert,
    select,
    update
)
from datetime import datetime

In [18]:
metadata = MetaData()

cookies = Table(
    "cookies",
    metadata,
    Column("cookie_id", Integer(), primary_key=True),
    Column("cookie_name", String(50), index=True),
    Column("cookie_recipe_url", String(255), nullable=False),
    Column("cookie_sku", String(55), nullable=False),
    Column("quantity", Integer()),
    Column("unit_cost", Numeric(12, 2)),
    CheckConstraint("quantity >= 0", name="quantity_positive")
)

users = Table(
    "users",
    metadata,
    Column("user_id", Integer(), primary_key=True),
    Column("username", String(50), unique=True, nullable=False),
    Column("email_address", String(255), nullable=False),
    Column("phone", String(20), nullable=False),
    Column("password", String(25), nullable=False),
    Column("created_on", DateTime(), default=datetime.now),
    Column("updated_on", DateTime, default=datetime.now, onupdate=datetime.now),
)

orders = Table(
    "orders",
    metadata,
    Column("order_id", Integer(), primary_key=True),
    Column("user_id", Integer(), ForeignKey("users.user_id")),
    Column("shipped", Boolean(), default=False),
)

line_items = Table(
    "line_items",
    metadata,
    Column("line_item_id", Integer(), primary_key=True),
    Column("order_id", ForeignKey("orders.order_id")),
    Column("cookie_id", ForeignKey("cookies.cookie_id")),
    Column("quantity", Integer()),
    Column("extended_cost", Numeric(12, 2))
)


In [3]:
engine = create_engine("sqlite:///:memory:")
metadata.create_all(engine)
connection = engine.connect()

In [4]:
# AttributeError

ins = insert(users).values(
                        username="cookiemon",
                        email_address="mon@cookie.com",
                        phone="111-111-1111",
                        password="password"
                        )
rp = connection.execute(ins)   

s = select([users.c.username])\
        .where(users.c.username=="cookiemon")
rp = connection.execute(s)
r = rp.first()
r.email_address  # not a column that was selected

AttributeError: Could not locate column in row for column 'email_address'

In [7]:
# IntegrityError

s = select([users.c.username]).where(users.c.username=="cookiemon")
rp = connection.execute(s)
print(rp.fetchall())

ins = insert(users).values(
                        username="cookiemon",
                        email_address="mon@cookie.com",
                        phone="111-111-1111",
                        password="password"
                        )
rp = connection.execute(ins)  # a user with the username cookiemon already exists


[('cookiemon',)]


IntegrityError: (sqlite3.IntegrityError) UNIQUE constraint failed: users.username
[SQL: INSERT INTO users (username, email_address, phone, password, created_on, updated_on) VALUES (?, ?, ?, ?, ?, ?)]
[parameters: ('cookiemon', 'mon@cookie.com', '111-111-1111', 'password', '2019-10-25 15:29:46.692988', '2019-10-25 15:29:46.693007')]
(Background on this error at: http://sqlalche.me/e/gkpj)

In [16]:
# Handling Errors

from sqlalchemy.exc import IntegrityError

ins = insert(users).values(
                        username="cookiemon",
                        email_address="mon@cookie.com",
                        phone="111-111-1111",
                        password="password"
                        )
try:
    connection.execute(ins)
except IntegrityError as error:
    print(error.args, error.params)

('(sqlite3.IntegrityError) UNIQUE constraint failed: users.username',) ('cookiemon', 'mon@cookie.com', '111-111-1111', 'password', '2019-10-25 15:51:00.667800', '2019-10-25 15:51:00.667809')


In [20]:
# Transactions

engine = create_engine("sqlite:///:memory:")  # new db
metadata.create_all(engine)
connection = engine.connect()

ins = insert(users).values(
                        username="cookiemon",
                        email_address="mon@cookie.com",
                        phone="111-111-1111",
                        password="password"
                        )
_ = connection.execute(ins)

ins = cookies\
    .insert()\
    .values(
        cookie_name="chocolate chip",
        cookie_recipe_url="http://some.url",
        cookie_sku="CC01",
        unit_cost="0.50",
        quantity="12"  # numeric
        )
rp = connection.execute(ins)
ins = insert(cookies)\
        .values(
            cookie_name="dark chocolate chip",
            cookie_recipe_url="http://some2.url",
            cookie_sku="CC02",
            unit_cost="0.75",
            quantity="1"  # numeric
        )
rp = connection.execute(ins)

In [21]:
ins = insert(line_items).values(
        order_id=1,
        cookie_id=1,
        quantity=9,
        extended_cost=4.50)
_ = connection.execute(ins)

to_insert = [
    {
        "order_id": 2,
        "cookie_id": 1,
        "quantity": 4,
        "extended_cost": 1.50,
    },
    {
        "order_id": 2,
        "cookie_id": 2,
        "quantity": 1,
        "extended_cost": 4.50,
    }
]
ins = insert(line_items)
result = connection.execute(ins, to_insert)

In [25]:
def ship_it(order_id):
    s = select([line_items.c.cookie_id,
                line_items.c.quantity])\
                .where(line_items.c.order_id == order_id)
    cookies_to_ship = connection.execute(s)
    for cookie in cookies_to_ship:
        u = update(cookies)\
                .where(cookies.c.cookie_id==cookie.cookie_id)\
                .values(quantity=cookies.c.quantity - cookie.quantity)
        result = connection.execute(u)
    print(f"Shipped order ID: {order_id}")

ship_it(1)
s = select([cookies.c.cookie_name, cookies.c.quantity])
rp = connection.execute(s)
rp.fetchall()

    

Shipped order ID: 1


[('chocolate chip', 3), ('dark chocolate chip', 10)]

In [28]:
def ship_it_safe(order_id):
    s = select([line_items.c.cookie_id,
                line_items.c.quantity])\
                .where(line_items.c.order_id == order_id)
    transaction = connection.begin()
    cookies_to_ship = connection.execute(s).fetchall()
    try:
        for cookie in cookies_to_ship:
            u = update(cookies)\
                    .where(cookies.c.cookie_id==cookie.cookie_id)\
                    .values(quantity=cookies.c.quantity - cookie.quantity)
            result = connection.execute(u)
        transaction.commit()
        print(f"Shipped order ID: {order_id}")
    except IntegrityError as error:
        transaction.rollback()
        print(error)

ship_it_safe(1)

(sqlite3.IntegrityError) CHECK constraint failed: quantity_positive
[SQL: UPDATE cookies SET quantity=(cookies.quantity - ?) WHERE cookies.cookie_id = ?]
[parameters: (9, 1)]
(Background on this error at: http://sqlalche.me/e/gkpj)
